In [19]:
import torch, librosa
from muq import MuQMuLan
device = 'cuda'
mulan = MuQMuLan.from_pretrained("OpenMuQ/MuQ-MuLan-large")
mulan = mulan.to(device).eval()

/user/zhouyz/anaconda3/envs/zhouyz/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [19]:
import json

example_json_path = "/data2/zhouyz/rec/MSD/MSD_2/save/AA/TRAAJEN144D1BB07C0.json"
example_music_path = "/data2/zhouyz/rec/MSD/MSD_2/save/AA/TRAAJEN144D1BB07C0.mp3"

with open(example_json_path, 'r') as f:
    example_json = json.load(f)


In [35]:
example_json

[{'name': 'I Like Boys - Remastered',
  'artists': ['Missing Persons'],
  'artist': 'Missing Persons',
  'genres': ['new wave', 'synthpop'],
  'disc_number': 1,
  'disc_count': 1,
  'album_name': 'Classic Masters',
  'album_artist': 'Missing Persons',
  'duration': 159,
  'year': 2002,
  'date': '2002-01-01',
  'track_number': 1,
  'tracks_count': 12,
  'song_id': '6lWxs8EyMDs7sbh5FfSk6h',
  'explicit': False,
  'publisher': 'Capitol Records',
  'url': 'https://open.spotify.com/track/6lWxs8EyMDs7sbh5FfSk6h',
  'isrc': 'USCA20200187',
  'cover_url': 'https://i.scdn.co/image/ab67616d0000b2739a9d2ce97905ea4b4db2b9d7',
  'copyright_text': '© 2002 Capitol Records, LLC',
  'download_url': 'https://music.youtube.com/watch?v=A-V74xsb2Mk',
  'lyrics': None,
  'popularity': 18,
  'album_id': '6wLpbnbRhOyVcqzH8yqGtq',
  'list_name': None,
  'list_url': None,
  'list_position': None,
  'list_length': None,
  'artist_id': '5CT7RBitS0e0u78T8tRBWQ',
  'album_type': 'album'}]

In [37]:
def get_decade_from_year(year):
    """
    将一个年份转换为年代的字符串格式，例如 2014 -> "10s"。
    
    参数:
    year (int): 年份。
    
    返回:
    str: 年代字符串。
    """
    if not isinstance(year, int) or year < 0:
        return "Invalid year"
    decade = (year % 100) // 10 * 10
    return f"{decade:02d}s"

In [38]:
wav, sr = librosa.load(example_music_path, sr = 24000)
wavs = torch.tensor(wav).unsqueeze(0).to(device) 
with torch.no_grad():
    audio_embeds = mulan(wavs = wavs)

texts = [', '.join(example_json[0]['genres']) + get_decade_from_year(example_json[0]['year'])] #+ ' title:' + example_json[0]['name']]

with torch.no_grad():
    text_embeds = mulan(texts = texts)


In [31]:
text_embeds.shape

torch.Size([1, 512])

In [39]:
sim = mulan.calc_similarity(audio_embeds, text_embeds)
print(sim)

tensor([[0.3326]], device='cuda:0')


In [13]:
import os
from tqdm import tqdm

empty_genres_count = 0
folder_path = '/data2/zhouyz/rec/MSD/'
for root, _, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.json'):
            file_path = os.path.join(root, file)
            with open(file_path, 'r') as f:
                data = json.load(f)
                if 'genres' in data[0]:
                    genres = data[0]['genres']
                    if len(genres) == 0:
                        empty_genres_count += 1
                        # print(f"File: {file} has empty 'genres' list.")
                    # print(f"File: {file}, Genres: {genres}")
                else:
                    print(f"File: {file} does not contain 'genres' key.")

print(f"Total files with empty 'genres' list: {empty_genres_count}")

Total files with empty 'genres' list: 24983


## For Music4all Format Data

In [33]:
import pandas as pd

m4a_items = pd.read_csv("/user/zhouyz/rec/recbole_v2/dataset/m4a/m4a.item", sep='\t')


In [41]:
m4a_items

,tracks_id:token,genres:token,lang:token,tags:float,spotify_id:float,popularity:float,release:float,danceability:float,energy:float,key:token,mode:token,valence:float,tempo:float,duration_ms:float,artist:float,song:float,album_name:float
0,0009fFIM1eYThaPg,pop,en,"pop,british,female vocalists,dance,cheryl cole",3eObKIfHKJ1nAPh0wTxFCc,12.0,2009,0.635,0.746,6.0,1.0,0.548,110.973,229947,Cheryl,Rain on Me,3 Words
1,0010xmHR6UICBOYT,underground hip hop,en,"instrumental hip-hop,underground hip hop,instr...",27szvF97Tu95GxN98N52fy,46.0,2013,0.591,0.513,7.0,0.0,0.263,172.208,325096,Oddisee,After Thoughts,The Beauty in All
2,002Jyd0vN4HyCpqL,"hard rock,rock,classic rock",en,"hard rock,rock,classic rock,american artist",273lBFpxUCwisTpdnF9cVb,31.0,1974,0.319,0.925,2.0,1.0,0.658,157.630,285693,Blue Öyster Cult,ME 262,Secret Treaties
3,006TYKNjNxWjfKjy,"symphonic metal,power metal,symphonic power metal",en,"symphonic metal,power metal,symphonic power metal",1qZgergQ41vaD4zBf3AKXR,33.0,2017,0.432,0.979,7.0,1.0,0.162,90.008,332867,Rhapsody,Flames of Revenge,Legendary Years (Re-Recorded)
4,007LIJOPQ4Sb98qV,"post-punk,new wave",en,"post-punk,new wave,1985",6rVxJ3sN3Cz40MSLavbG1K,19.0,2009,0.357,0.708,9.0,1.0,0.470,123.904,326067,The Chameleons,Nostalgia,What Does Anything Mean? Basically (2009 Remas...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109264,zzyyPUs7hC9Nz2e1,"hardcore punk,punk,hardcore",en,"hardcore punk,punk,hardcore,punk rock",6IW5yPlWJYmXxRuuc7QkMx,36.0,2017,0.380,0.835,1.0,1.0,0.539,142.699,200880,Frank Carter & The Rattlesnakes,Vampires,Modern Ruin
109265,zzz0n04uuTUA7fNh,pop,en,"pop,&lt;3,omg",21pCToQdqcPi879auxjxKe,48.0,2014,0.572,0.503,6.0,0.0,0.553,103.215,262840,Justin Bieber,Heartbreaker,Journals
109266,zzzj3LYaZtYtbzSr,singer-songwriter,en,"female vocalists,indie,singer-songwriter,piano...",2SJ6FGmRPacKE9N4g4BRZU,21.0,2010,0.402,0.376,2.0,0.0,0.119,108.509,276507,Regina Spektor,Après Moi - Live In London,Live In London
109267,zzznMjZAKnJJXQSj,"synthpop,remix,pop",en,"synthpop,80s,remix,pop",2ygfHXyt3gvyhvKrNJU61n,47.0,2017,0.532,0.977,10.0,0.0,0.966,116.527,224549,Dua Lipa,New Rules (Initial Talk Remix),New Rules (Initial Talk Remix)


In [ ]:
from tqdm import tqdm
import numpy as np
output_path = '/user/zhouyz/rec/recbole_v2/dataset/m4a/m4a-muqmulan-texts/'
os.makedirs(output_path, exist_ok=True)
for row in tqdm(m4a_items.iterrows()):
    track_id = row[1]['tracks_id:token']
    tags = row[1]['tags:float'].split(',')
    language = row[1]['lang:token']
    texts = [', '.join(tags) + ', ' + language]
    with torch.no_grad():
        text_embeds = mulan(texts = texts)
    text_embeds = text_embeds.cpu().numpy().squeeze(0)
    assert text_embeds.shape == (512,), f"Expected shape (1024,), got {text_embeds.shape}"
    output_file = os.path.join(output_path, track_id + '.npy')
    np.save(output_file, text_embeds)



117it [00:02, 39.55it/s]


KeyboardInterrupt: 

## LFM2b Format

In [ ]:
import pandas as pd

# 逐行读取文件
data = []
# 假设列名为 col1, col2, col3, col4, tags
# 您可以根据实际情况修改列名

with open("/user/zhouyz/rec/recbole_v2/dataset/lfm2b/tags_all_music.tsv", 'r', encoding='utf-8') as f:
    column_names = f.readline().strip().split('\t')[:4] + ['tags']
    for line in f:
        parts = line.strip().split('\t')
        if len(parts) >= 4:
            # 前四列为固定列
            row_data = parts[:4]
            # 第五列及之后的所有部分合并为tags列表
            tags = parts[4:]
            row_data.append(tags)
            data.append(row_data)
        # 您可以根据需要处理少于4列的行
        # elif parts and parts[0]: 
        #     data.append([parts[0], '', '', '', []])


# 创建DataFrame
tags_df = pd.DataFrame(data, columns=column_names)
tags_df

,track_id,artist,track,count,tags
0,36346257,Gotye,Somebody That I Used to Know,284021,"[electronic, pop, rock]"
1,32496162,Imagine Dragons,Radioactive,233427,"[rock, electronic, pop]"
2,33619193,Adele,Rolling in the Deep,223400,"[pop, blues, jazz, rock, rnb, vocal]"
3,32083560,Foster the People,Pumped Up Kicks,211291,"[pop, electronic, rock, holiday]"
4,45222862,Pink Floyd,Wish You Were Here,211148,"[rock, classical, pop]"
...,...,...,...,...,...
1874813,35182113,U-Roy,Sharper Than Joe Fraser,1,[reggae]
1874814,19273327,Chris Ardoin,I Don't Want Nobody Here But You,1,[blues]
1874815,27785946,Crystal Distortion,NY2023,1,[electronic]
1874816,27668260,Pod,Myopsis (Dark Comedy Mix),1,[electronic]


In [20]:
from tqdm import tqdm
import numpy as np
import os 
from tqdm import tqdm

output_path = '/user/zhouyz/rec/recbole_v2/dataset/lfm2b/lfm2b-muqmulan-texts/'
os.makedirs(output_path, exist_ok=True)
for row in tqdm(tags_df.iterrows()):
    track_id = row[1]['track_id']
    tags = row[1]['tags']

    texts = [', '.join(tags)]
    with torch.no_grad():
        text_embeds = mulan(texts = texts)
    text_embeds = text_embeds.cpu().numpy().squeeze(0)
    assert text_embeds.shape == (512,), f"Expected shape (1024,), got {text_embeds.shape}"
    output_file = os.path.join(output_path, track_id + '.npy')
    np.save(output_file, text_embeds)



251it [00:07, 32.76it/s]


KeyboardInterrupt: 

In [ ]:
from tqdm import tqdm
import numpy as np
output_path = '/user/zhouyz/rec/recbole_v2/dataset/m4a/m4a-muqmulan-texts/'
os.makedirs(output_path, exist_ok=True)
for row in tqdm(m4a_items.iterrows()):
    track_id = row[1]['tracks_id:token']
    tags = row[1]['tags:float'].split(',')
    language = row[1]['lang:token']
    texts = [', '.join(tags) + ', ' + language]
    with torch.no_grad():
        text_embeds = mulan(texts = texts)
    text_embeds = text_embeds.cpu().numpy().squeeze(0)
    assert text_embeds.shape == (512,), f"Expected shape (1024,), got {text_embeds.shape}"
    output_file = os.path.join(output_path, track_id + '.npy')
    np.save(output_file, text_embeds)

